Construindo um grafo automaticamente
====================================



## Introdução



Neste notebook nós vamos dar o primeiro passo para construir nossa rede neural artificial. Neste primeiro passo, nós vamos criar uma classe que gera automaticamente o nosso `grafo computacional`. O grafo computacional é o grafo que representa todas as operações matemáticas que ocorreram ao se computar um certo valor $y$. O grafo computacional é um passo necessário pois será baseado nele que iremos computar os gradientes locais necessários para realizar o `backpropagation`.



## Importações



In [1]:
from funcoes import plota_grafo

## Código e discussão



### Primeiros passos



A base de tudo será uma classe chamada `Valor`. Vamos começar pelo básico!



In [2]:
class Valor:
    def __init__(self, data):
        self.data = data

    def __repr__(self):
        return f"Valor(data={self.data})"

Vamos testar nossa classe!



In [3]:
x1 = Valor(60)
print(x1)
print(x1.data)

Valor(data=60)
60


### Os dunders `__add__` e `__mul__`



Observe que não conseguimos adicionar nem multiplicar objetos criados com a classe `Valor`.



In [4]:
a = Valor(10)
b = Valor(5)

In [5]:
print(a + b)

TypeError: unsupported operand type(s) for +: 'Valor' and 'Valor'

In [7]:
print(a * b)

TypeError: unsupported operand type(s) for *: 'Valor' and 'Valor'

Ué&#x2026; porque não conseguimos? Não conseguimos pois o Python (ainda) não é vidente. Ele lá vai saber como adicionar ou multiplicar algo que você criou? Pra você parece óbvio que valores podem ser adicionados ou multiplicados, mas para o Python ele nem sabe o que significa a palavra `Valor`&#x2026;

Como sempre, temos que contar para o programa o que queremos que aconteça quando usarmos os operadores `+` e `*`. Quem faz isso são os dunders `__add__` e `__mul__`.



In [6]:
class Valor:
    def __init__(self, data):
        self.data = data

    def __repr__(self):
        return f"Valor(data={self.data})"

    def __add__(self, outro_valor):
        saida = Valor(self.data + outro_valor.data)
        return saida

    def __mul__(self, outro_valor):
        saida = Valor(self.data * outro_valor.data)
        return saida

Vamos testar!



In [8]:
a = Valor(10)
b = Valor(5)
print(a + b)
print(a * b)

Valor(data=15)
Valor(data=50)


### Registrando os progenitores



Nosso objetivo é construir um grafo computacional. Em um grafo computacional, um certo vértice pode ter um ou mais vértices progenitores (são seus pais/mães). Nós não podemos perder essa informação quando formos construir um grafo, então precisamos incluir essa informação na nossa classe.



In [10]:
class Valor:
    def __init__(self, data, progenitor=()):
        self.data = data
        self.progenitor = progenitor

    def __repr__(self):
        return f"Valor(data={self.data})"

    def __add__(self, outro_valor):
        data = self.data + outro_valor.data
        progenitor = (self, outro_valor)
        saida = Valor(data, progenitor)
        return saida

    def __mul__(self, outro_valor):
        data = self.data * outro_valor.data
        progenitor = (self, outro_valor)
        saida = Valor(data, progenitor)
        return saida

Vamos testar!



In [15]:
a = Valor(10)
b = Valor(5)

c = a+b
d = a * b
print(c.progenitor)
print(d.data)

(Valor(data=10), Valor(data=5))
50


### Registrando o operador mãe



Em um grafo computacional, um vértice pode ter um operador mãe. O operador mãe é o operador que foi usado para gerar o vértice.



In [16]:
class Valor:
    def __init__(self, data, progenitor=(), operador_mae=""):
        self.data = data
        self.progenitor = progenitor
        self.operador_mae = operador_mae

    def __repr__(self):
        return f"Valor(data={self.data})"

    def __add__(self, outro_valor):
        data = self.data + outro_valor.data
        progenitor = (self, outro_valor)
        operador_mae = "+"
        saida = Valor(data, progenitor, operador_mae)
        return saida

    def __mul__(self, outro_valor):
        data = self.data * outro_valor.data
        progenitor = (self, outro_valor)
        operador_mae = "*"
        saida = Valor(data, progenitor, operador_mae)
        return saida

Vamos testar!



In [18]:
a = Valor(10)
b = Valor(5)

c = a+b
d = a * b
print(c.progenitor)
print(d.operador_mae)

(Valor(data=10), Valor(data=5))
*


### Plotando o primeiro grafo



Vamos plotar nosso primeiro grafo!



In [22]:
a = Valor(2)
b = Valor(-3)
c = Valor(10)

d = a * b
e = d + c

grafo = plota_grafo(e)
print(grafo)

digraph {
	graph [rankdir=LR]
	3120114776096 [label="{  | data -3.0000 }" shape=record]
	3120113612928 [label="{  | data 2.0000 }" shape=record]
	3120113613648 [label="{  | data 4.0000 }" shape=record]
	"3120113613648+" [label="+"]
	"3120113613648+" -> 3120113613648
	3120113611632 [label="{  | data 10.0000 }" shape=record]
	3120113614752 [label="{  | data -6.0000 }" shape=record]
	"3120113614752*" [label="*"]
	"3120113614752*" -> 3120113614752
	3120114776096 -> "3120113614752*"
	3120113611632 -> "3120113613648+"
	3120113614752 -> "3120113613648+"
	3120113612928 -> "3120113614752*"
}



### Registrando o rótulo



Nosso grafo seria mais legível se tivéssemos rótulos indicando o que é cada vértice. Vamos incluir essa informação na nossa classe.



In [20]:
class Valor:
    def __init__(self, data, progenitor=(), operador_mae="", rotulo=""):
        self.data = data
        self.progenitor = progenitor
        self.operador_mae = operador_mae
        self.rotulo = rotulo

    def __repr__(self):
        return f"Valor(data={self.data})"

    def __add__(self, outro_valor):
        data = self.data + outro_valor.data
        progenitor = (self, outro_valor)
        operador_mae = "+"
        saida = Valor(data, progenitor, operador_mae)
        return saida

    def __mul__(self, outro_valor):
        data = self.data * outro_valor.data
        progenitor = (self, outro_valor)
        operador_mae = "*"
        saida = Valor(data, progenitor, operador_mae)
        return saida

Vamos testar!



In [23]:
a = Valor(2, rotulo="a")
b = Valor(-3, rotulo="b")
c = Valor(10, rotulo="c")

d = a * b
e = d + c

d.rotulo = "d"
e.rotulo = "e"

plota_grafo(e)
grafo = plota_grafo(e)
print(grafo)

digraph {
	graph [rankdir=LR]
	3120113613408 [label="{ d | data -6.0000 }" shape=record]
	"3120113613408*" [label="*"]
	"3120113613408*" -> 3120113613408
	3120112269968 [label="{ b | data -3.0000 }" shape=record]
	3120113614224 [label="{ e | data 4.0000 }" shape=record]
	"3120113614224+" [label="+"]
	"3120113614224+" -> 3120113614224
	3120113121696 [label="{ a | data 2.0000 }" shape=record]
	3120113120208 [label="{ c | data 10.0000 }" shape=record]
	3120113613408 -> "3120113614224+"
	3120112269968 -> "3120113613408*"
	3120113120208 -> "3120113614224+"
	3120113121696 -> "3120113613408*"
}



### Refazendo o grafo que fizemos na aula anterior



Na aula anterior nós fizemos um grafo computacional para aprender como funciona o backpropagation. Vamos refazer ele aqui!



In [24]:
x1 = Valor(60, rotulo = "x1")
x2 = Valor(24, rotulo = "x2")
w1 =Valor(10, rotulo = "w1")
w2 = Valor(5, rotulo = "w2")
w3 = Valor(2, rotulo = "w3")
b= Valor(7, rotulo = "b")

s1 = x1*w1
s1.rotulo = 's1'

s2 = x2*w2
s2.rotulo = 's2'

n = s1+s2
n.rotulo = 'n'

k = n+b
k.rotulo = 'k'

y= k*w3
y.rotulo = 'y'

### discussão

Continuamos com o uso de classes para representar nosso conjunto de dados. Nesse tipo de armazenamento de informações temos algumas funções já conhecidas, nessa aula aprendemos os métodos `__add__` e `__mul__`, que como vimos no notebook anterior são chamadas de dunders. O método `__add__` é usado para adicionar os valores que o seu objeto possui e o `__mul__` irá multiplicar. Cada um desses dunders usam um operador especifico, o `__add__` usa o `+` e o `__mul__` usa o `*`, eles realmente fazem as operações matemáticas cujos simbolos nos remetem.


No segunod momento dessa aula, registramos as ações que nosso métodos executaram por meio de um grafo. O primeiro passo foi regristrar os progenitores, os "pais" dos nossos próximos valores, depois regr

## Conclusão

Na continuação do exercicio anterior, nesse experimento pudemos botar em prática todos assuntos discutidos na primeira aula teorica de redes neurais. Aqui aprendemos os raciocinios de uma classe e quando devemos usa-las. Nesse exercicio, criamos uma classe que gerou automaticamente o nosso grafo computacional. O grafo computacional era o grafo que representava todas as operações matemáticas que ocorreram ao se computar um certo valor 𝑦.Nos atentamos aos grafos pois serão com eles que faremos o backpropagation.


Observações durante a aula:
- Um objeto novo é uma instancia, ou seja, o estado do objeto em um determinado momento/ etapa do código.

## Playground

